In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.model_selection import train_test_split
# from datasets import Dataset


In [ ]:
df = pd.read_csv('Data_with_description_9444.csv')
df.head()

,image_link,group_id,entity_name,entity_value,description
0,https://m.media-amazon.com/images/I/51tZoSQjQZ...,101697.0,width,40.0 centimetre,50 cm 40 cm
1,https://m.media-amazon.com/images/I/612f4sf+uM...,101697.0,depth,28.0 inch,Thicker than Others Others Ours 0.4mm 1mm Lar...
2,https://m.media-amazon.com/images/I/619S00V9Wp...,101697.0,depth,20.0 centimetre,"TALLAGRANDE 7.8/20cm 2""/5cm 6.89""/17.5cm"
3,https://m.media-amazon.com/images/I/51iQwQjQ+7...,101697.0,width,5.0 inch,"Happy Birthday! 4n 5"" 6"" uL 8"" 9"" 10"" 11"" 12"""
4,https://m.media-amazon.com/images/I/514j9sXE6S...,101697.0,depth,27.2 centimetre,23.3cm/9.17in 27.2cm/10.7in


In [ ]:
df['entity_name'].unique()

array(['width', 'depth', 'height', 'item_weight',
       'maximum_weight_recommendation', 'item_volume', 'wattage',
       'voltage', nan], dtype=object)

In [ ]:
df['image_link'][4]

'https://m.media-amazon.com/images/I/514j9sXE6SL.jpg'

In [ ]:
len(df)

11031

In [ ]:
df['group_id'].value_counts()

,count
group_id,
752266,481
120219,402
459516,254
318770,254
271537,190
...,...
656506,2
351545,2
997333,2


In [ ]:
df1 = df.drop('image_link', axis=1)
df1.head()

,group_id,entity_name,entity_value,description
0,101697,width,40.0 centimetre,50 cm 40 cm
1,101697,depth,28.0 inch,Thicker than Others Others Ours 0.4mm 1mm Lar...
2,101697,depth,20.0 centimetre,"TALLAGRANDE 7.8/20cm 2""/5cm 6.89""/17.5cm"
3,101697,width,5.0 inch,"Happy Birthday! 4n 5"" 6"" uL 8"" 9"" 10"" 11"" 12"""
4,101697,depth,27.2 centimetre,23.3cm/9.17in 27.2cm/10.7in


In [ ]:
def prepare_input(row):
    return f"group: {row['group_id']} entity_name: {row['entity_name']} description: {row['description']}"

df1['input'] = df1.apply(prepare_input, axis=1)
df1['output'] = df1['entity_value']

In [ ]:
df1.head()

,group_id,entity_name,entity_value,description,input,output
0,101697,width,40.0 centimetre,50 cm 40 cm,group: 101697 entity_name: width description: ...,40.0 centimetre
1,101697,depth,28.0 inch,Thicker than Others Others Ours 0.4mm 1mm Lar...,group: 101697 entity_name: depth description: ...,28.0 inch
2,101697,depth,20.0 centimetre,"TALLAGRANDE 7.8/20cm 2""/5cm 6.89""/17.5cm",group: 101697 entity_name: depth description: ...,20.0 centimetre
3,101697,width,5.0 inch,"Happy Birthday! 4n 5"" 6"" uL 8"" 9"" 10"" 11"" 12""",group: 101697 entity_name: width description: ...,5.0 inch
4,101697,depth,27.2 centimetre,23.3cm/9.17in 27.2cm/10.7in,group: 101697 entity_name: depth description: ...,27.2 centimetre


In [ ]:
train_df, val_df = train_test_split(df1, test_size=0.2, random_state=42)

In [ ]:
model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)


In [ ]:
train_dataset

Dataset({
    features: ['group_id', 'entity_name', 'entity_value', 'description', 'input', 'output', '__index_level_0__'],
    num_rows: 8824
})

In [ ]:
def tokenize_function(examples):
    inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(examples["output"], padding="max_length", truncation=True, max_length=128)
    return {
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
        "labels": outputs.input_ids
    }

In [ ]:
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_val = val_dataset.map(tokenize_function, batched=True, remove_columns=val_dataset.column_names)


Map:   0%|          | 0/8824 [00:00<?, ? examples/s]

Map:   0%|          | 0/2207 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Added to match evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    fp16=True,
    gradient_accumulation_steps=4,
    save_total_limit=2,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # Specify which metric to use for selecting the best model
)

In [ ]:
from transformers import DataCollatorForSeq2Seq

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,0.054100,0.041598
2,0.044200,0.037206


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=1653, training_loss=0.2841743377217075, metrics={'train_runtime': 2371.1158, 'train_samples_per_second': 11.164, 'train_steps_per_second': 0.697, 'total_flos': 1.610571828953088e+16, 'train_loss': 0.2841743377217075, 'epoch': 2.99728014505893})

In [ ]:
trainer.save_model("./fine_tuned_model")


In [ ]:
input_text = "group: 101697 entity_name: width description: Happy Birthday! 4n 5\" 6\" uL 8\" 9\" 10\" 11\" 12\""
inputs = tokenizer(input_text, return_tensors="pt")
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}
outputs = model.generate(**inputs)
predicted_entity_value = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Predicted entity value: {predicted_entity_value}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predicted entity value: 5.0 inch


In [ ]:
def predict(input_text):
    inputs = tokenizer(input_text, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
test_input = "group: 101697 entity_name: depth description: 23.3cm/9.17in 27.2cm/10.7in"
prediction = predict(test_input)
print(f"Input: {test_input}")
print(f"Predicted entity value: {prediction}")

Input: group: 101697 entity_name: depth description: 23.3cm/9.17in 27.2cm/10.7in
Predicted entity value: 27.0 centimetre


In [ ]:
eval_results = trainer.evaluate()


In [ ]:
eval_results

{'eval_loss': 0.03720603138208389,
 'eval_runtime': 66.0455,
 'eval_samples_per_second': 33.416,
 'eval_steps_per_second': 8.358,
 'epoch': 2.99728014505893}